# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [65]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [66]:
#getting data: the dataset was uploaded to and is accesible through my github repo
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
hyperpdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [67]:
ws = Workspace.from_config()
expname = 'hyperp3ht'

exphyper=Experiment(ws, expname)

In [68]:
# Compute name has restrictions, please refer to official documentation
ccname = "ccp03hyp"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists already")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [76]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
#param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4,5), "--max_iter": choice(80,100,120,150,170,200)})
param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4), "--max_iter": choice(40,80,120,130,200)})

if "outputs" not in os.listdir():
    os.mkdir("./outputs")

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target=ccluster, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, policy=early_termination_policy, primary_metric_name="Accuracy",
                                          hyperparameter_sampling=param_sampling,
                                         max_total_runs=20,
                                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [77]:
#TODO: Submit your experiment
hyperDrive_run = exphyper.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [78]:
RunDetails(hyperDrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [79]:
brmodel = hyperDrive_run.get_best_run_by_primary_metric()
brmetrics = brmodel.get_metrics()

print('Best Run Id: ', brmodel.id)
print('\n Accuracy:', brmetrics['Accuracy'])
print('\n Regularization Strength:',brmetrics['Regularization Strength:'])
print('\n Max Iterations:',brmetrics['Max iterations:'])

Best Run Id:  HD_14a6b373-ad69-4918-88b5-f937e35a5d59_11

 Accuracy: 0.8914342629482072

 Regularization Strength: 3.0

 Max Iterations: 40


In [80]:
#this lines is really useful in case you want to check files related to the model
brmodel.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_e731c75ab958506373e89d8bf408b534affca52f464bf231de4b9f7536da898f_d.txt',
 'azureml-logs/65_job_prep-tvmps_e731c75ab958506373e89d8bf408b534affca52f464bf231de4b9f7536da898f_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_e731c75ab958506373e89d8bf408b534affca52f464bf231de4b9f7536da898f_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/101_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/bhypermodel.pkl']

In [81]:
#TODO: Save the best model: the files is created and uploaded during the execution of train.py
model = brmodel.register_model(model_name = 'bhypermodel', model_path = 'outputs/bhypermodel.pkl') 
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-136043', subscription_id='5a4ab2ba-6c51-4805-8155-58759ad589d8', resource_group='aml-quickstarts-136043'), name=bhypermodel, id=bhypermodel:6, version=6, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [82]:
from azureml.core import Environment
env = Environment.get(workspace=ws, name="AzureML-AutoML")
#OPCIONAL: env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)

In [83]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model

infcfg = InferenceConfig(entry_script='score.py',
                                    environment=env)

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wshypp03ht01", [model], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://1c363841-61d1-45f9-9797-06cb1a0def5d.southcentralus.azurecontainer.io/score
http://1c363841-61d1-45f9-9797-06cb1a0def5d.southcentralus.azurecontainer.io/swagger.json


In [84]:
import json

#Importing the dataset for testing 
tsthomic2018 = hyperpdhomic2018.sample(10) 
lblhomic2018 = tsthomic2018.pop('sexo')

#tstdatahomic2018 = json.dumps({'data': tsthomic2018.to_dict(orient='records')})
#print(tstdatahomic2018)

data = {"data":
        [
          {
           "id": 2125, "regfalle": 21917, "edad": 42, "deptoocuhe": 2, "tipoarma": 1, "pracaut": 1
          },
      ]
    }
         
# Convert to JSON string
tstdatahomic2018 = json.dumps(data)

print(tstdatahomic2018)

{"data": [{"id": 2125, "regfalle": 21917, "edad": 42, "deptoocuhe": 2, "tipoarma": 1, "pracaut": 1}, {"id": 755, "regfalle": 19621, "edad": 22, "deptoocuhe": 5, "tipoarma": 1, "pracaut": 1}, {"id": 2491, "regfalle": 24224, "edad": 43, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 1}, {"id": 2443, "regfalle": 20976, "edad": 20, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 1}, {"id": 2170, "regfalle": 23144, "edad": 49, "deptoocuhe": 4, "tipoarma": 1, "pracaut": 0}, {"id": 1574, "regfalle": 23107, "edad": 40, "deptoocuhe": 5, "tipoarma": 1, "pracaut": 1}, {"id": 2789, "regfalle": 24235, "edad": 30, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 1}, {"id": 1377, "regfalle": 16494, "edad": 25, "deptoocuhe": 6, "tipoarma": 9, "pracaut": 0}, {"id": 2403, "regfalle": 23280, "edad": 38, "deptoocuhe": 1, "tipoarma": 1, "pracaut": 1}, {"id": 2037, "regfalle": 20709, "edad": 14, "deptoocuhe": 6, "tipoarma": 1, "pracaut": 0}]}


TODO: In the cell below, send a request to the web service you deployed to test it.

In [85]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [86]:
# Print results
print(response.text)

"float() argument must be a string or a number, not 'dict'"


In [87]:
# Print original labels
print(lblhomic2018)

2124    0
754     0
2490    1
2442    0
2169    0
1573    0
2788    0
1376    0
2402    0
2036    0
Name: sexo, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [88]:
print(webservice.get_logs())

2021-01-26T01:39:43,543454304+00:00 - iot-server/run 
2021-01-26T01:39:43,543730523+00:00 - rsyslog/run 
2021-01-26T01:39:43,545348130+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-26T01:39:43,552748722+00:00 - gunicorn/run 
rsyslogd

In [91]:
#Optional: converting the model to ONNX format
#brmodel2 = hyperDrive_run.get_output()
#brmodel2 = hyperDrive_run.get_best_run_by_primary_metric()
#from azureml.automl.runtime.onnx_convert import OnnxConverter

#brmodel2, onnx_model= hyperDrive_run.get_output(return_onnx_model=True)
#brmodel2, onnx_model= hyperDrive_run.get_best_run_by_primary_metric(return_onnx_model=True)
#OnnxConverter.save_onnx_model(onnx_model, file_path="./hyperbmodel.onnx")
#print(brmodel2)

#pip install skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType

model_onnx = convert_sklearn(pipeline,
                             "quality",
                             initial_types=[("input", StringTensorType([None, 1]))])

with open("pipeline_quality.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())


TypeError: get_best_run_by_primary_metric() got an unexpected keyword argument 'return_onnx_model'

In [ ]:
#BEST PRACTICES: deleting webservice and CCluster that won't be used anymore
webservice.delete()
ccluster.delete()